In [15]:
import pandas as pd


In [16]:
# Variables
data_files = ["data/JC-202301-citibike-tripdata.csv",
              "data/JC-202302-citibike-tripdata.csv",
              "data/JC-202303-citibike-tripdata.csv",
              "data/JC-202304-citibike-tripdata.csv",
              "data/JC-202305-citibike-tripdata.csv",
              "data/JC-202306-citibike-tripdata.csv",
              "data/JC-202307-citibike-tripdata.csv",
              "data/JC-202308-citibike-tripdata.csv",
              "data/JC-202309-citibike-tripdata.csv",
              "data/JC-202310-citibike-tripdata.csv",
              "data/JC-202311-citibike-tripdata.csv",
              "data/JC-202312-citibike-tripdata.csv"              
              ]

In [17]:
# Combines all of the CSV's into a single dataframe
def combine_data(data_files):
    bike_data = pd.DataFrame()
    for x in range(0,12):
        print(f"Adding:  {data_files[x]}")
        data = pd.read_csv(data_files[x], encoding="UTF-8")
        bike_data = pd.concat([bike_data, data], ignore_index=True)
    print()
    return(bike_data)

In [18]:
# Saves final dataset
def save_data(bike_data):
    print("Saving CSV.")
    bike_data.to_csv("data/combined_data.csv", index=False, header=True)
    print("Save complete.")

In [19]:
# Adds duration column to dataframe
def add_duration(bike_data):
    # convert columns to date/time
    bike_data["started_at"] = pd.to_datetime(bike_data["started_at"])
    bike_data["ended_at"] = pd.to_datetime(bike_data["ended_at"])

    # Find the duration in minutes, rounded to 2 decimal points
    bike_data['duration_minutes'] = (bike_data['ended_at'] - bike_data['started_at']).dt.total_seconds() / 60
    bike_data['duration_minutes'] = bike_data['duration_minutes'].round(2)

    return(bike_data)

In [20]:
def remove_outliers(bike_data):
    # removes any duration below 1 minute
    bike_data.drop(bike_data[bike_data['duration_minutes'] < 1].index, inplace = True) 

    # Find outliers to remove them
    quartiles = bike_data["duration_minutes"].quantile([.25, .5, .75])
    lowerq = quartiles[.25]
    upperq = quartiles[.75]
    iqr = upperq-lowerq
    lower_bound = lowerq - (1.5*iqr)
    upper_bound = upperq + (1.5*iqr)

    print(f"Duration Outliers")
    print(f"-----------------------------")
    print(f"Lower quartile: {lowerq}")
    print(f"Upper quartile: {upperq}")
    print(f"Interquartle range: {iqr}")
    print(f"Values below {lower_bound} could be outliers.")
    print(f"Values above {upper_bound} could be outliers.")
    print(f"Min:  {bike_data['duration_minutes'].min()}")
    print(f"Max:  {bike_data['duration_minutes'].max()}")
    print()

    # Removes outliers
    bike_data.drop(bike_data[(bike_data['duration_minutes'] < lower_bound) | (bike_data['duration_minutes'] > upper_bound)].index, inplace=True)

    return(bike_data)


In [21]:
# Init
bike_data = combine_data(data_files)
bike_data = add_duration(bike_data)

start_rows = len(bike_data)


bike_data = remove_outliers(bike_data)

end_rows = len(bike_data)

print("-----------------------------")
print(f"Starting Rows:  {start_rows}  After Cleaning:  {end_rows}  Rows Removed:  {start_rows - end_rows}")
print()
save_data(bike_data)


Adding:  data/JC-202301-citibike-tripdata.csv
Adding:  data/JC-202302-citibike-tripdata.csv
Adding:  data/JC-202303-citibike-tripdata.csv
Adding:  data/JC-202304-citibike-tripdata.csv
Adding:  data/JC-202305-citibike-tripdata.csv
Adding:  data/JC-202306-citibike-tripdata.csv
Adding:  data/JC-202307-citibike-tripdata.csv
Adding:  data/JC-202308-citibike-tripdata.csv
Adding:  data/JC-202309-citibike-tripdata.csv
Adding:  data/JC-202310-citibike-tripdata.csv
Adding:  data/JC-202311-citibike-tripdata.csv
Adding:  data/JC-202312-citibike-tripdata.csv

Duration Outliers
-----------------------------
Lower quartile: 4.2
Upper quartile: 10.33
Interquartle range: 6.13
Values below -4.995 could be outliers.
Values above 19.525 could be outliers.
Min:  1.0
Max:  17112.15

-----------------------------
Starting Rows:  988851  After Cleaning:  875771  Rows Removed:  113080

Saving CSV.
Save complete.
